# Ejemplo práctico. Audiolibros

### Problema

Nos han dado datos obtenidos con una App para Audiolibros.  Lógicamente, tiene que ver únicamente con las versiones audio de libros.  Cada cliente en la base de datos ha hecho una compra por lo menos una vez, por eso está en la base de datos.  Deseamos crear un algoritmo de ML que nos permita predecir si un cliente volverá a hacer una compra a la empresa de Audiolibros.

La idea principal es que si un cliente tiene una baja probabilidad de regresar, no hay razón alguna para gastar dinero en hacerles anunicios a él/ella.  Si podemos enfocar nuestros esfuerzos SOLAMENTE en aquellos clientes que probablemente compren de nuevo, podemos hacer grandes ahorros.  Es más, este modelo puede identificar las métricas importantes para que un cliente regrese.  La identificación de nuevos clientes crea valor y oportunidades de crecimiento.

Se tiene un .csv que resume los datos.  Hay varias variables:

* Customer ID, 
* Book length overall (suma de la duración en minutos de todas las compras), 
* Book length avg (duración promedio, en minutos, de todas las compras), 
* Price paid_overall (suma del precio de todas las compras),
* Price Paid avg (precio promedo de todas las compras), 
* Review (una variable Booleana indicando si el cliente dejó una evaluación), 
* Review out of 10 (si el/la cliente dejó una evaluación, su calificación de 0 a 10), 
* Total minutes listened (total de minutos escuchados), 
* Completion (completación de 0 a 1), 
* Support requests (número de requerimientos de soporte técnico), y 
* Last visited minus purchase date (Ultima visita menos la fecha de compra, en días)

Estas son las entradas (excluyendo Customer ID, ya que este es completamente arbitrario.  Es más como un nombre que un número).

Las metas son una variable Booleana (0 ó 1).  Tomaremos un período de 2 años para nuestras entradas, y los siguientes 6 meses como metas.  De esta forma, en realidad estamos prediciendo si:  basado en los últimos 2 años de actividad e involucramiento, un cliente volverá a comprar dentro de los siguientes 6 meses.  Seis meses suena con un tiempo razonable.  Si no compran luego de 6 meses, lo más probable es que se han ido a la competencia o no les gustó el formato de Audiolibros para recibir información.

La tarea es simple:  crear un algoritmo de ML, que pueda predecir si un cliente volverá a comprar.

Este es un problema de clasificación con dos clases:  "no comprará" o "comprará", representado por 0s y 1s. 

## Crear el algoritmo de ML



### Importar las librerías relevantes 

In [ ]:
import numpy as np
import tensorflow as tf

### Datos

Se cargarán los datos almacenados luego del pre-procesamiento (ver Notebook aparte)

Se crea una variable temporal *npz* para cargar los archivos, y que luego se utilizará para separar las entradas y metas de cada subconjunto.  Para esto se utilizará la palabra clave que se usó para guardarlos "entradas" o "metas", como índice.

Se asegurará que las entradas sean de tipo float y las metas de tipo int.



In [ ]:
npz = np.load('Audiolibros_datos_entreno.npz')
entradas_entreno = npz['entradas'].astype(float)
metas_entreno = npz['metas'].astype(int)

npz = np.load('Audiolibros_datos_validacion.npz')
entradas_validacion, metas_validacion = npz['entradas'].astype(float), npz['metas'].astype(int)

npz = np.load('Audiolibros_datos_prueba.npz')
entradas_prueba, metas_prueba = npz['entradas'].astype(float), npz['metas'].astype(int)

### Modelo

Bosquejo (diagrama), optimizadores, pérdida, detención temprana y entrenamiento

De lo anterior, se ha visto todo menos lo de detención (parada) temprana.  Este es un mecanismo para detener las iteraciones, una vez se detecta que la pérdida de validación empieza a aumentar.  

Se hace a través del método "callback" de **keras**.  El método callback permite ejecutar una rutina al completar un proceso, en este caso usaremos el **EarlyStopping** que detiene las iteraciones al detectar un aumento en la pérdida de validación.  El default es al primer aumento.  Sin embargo, se puede configurar con el argumento **patience**, que representa el número consecutivo de aumentos que debe detener el proceso.  Como es posible que hayan variaciones aleatorias en la pérdida, se usará un valor de 2...cuando haya dos aumentos consecutivos se detiene la iteración.



In [ ]:
tamanio_entrada = 10
tamanio_salida = 2
tamanio_capa_escondida = 50
    
modelo = tf.keras.Sequential([

    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida
   
    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])


modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


tamanio_tanda = 100

epocas_max = 100


parada_temprana = tf.keras.callbacks.EarlyStopping(patience = 2)


modelo.fit(entradas_entreno,
          metas_entreno, 
          batch_size = tamanio_tanda,
          epochs = epocas_max, # asumiendo que no entra en acción la detención temprana
#          callbacks=[parada_temprana], 
          validation_data=(entradas_validacion, metas_validacion), 
          verbose = 2 
          )  

## Probar el modelo

Luego de entrenar el modelo con los datos de entrenamiento, y validar con los datos de validación, debemos probar el potencial predictivo final del modelo.  Esto se hace ejecutando el modelo con los datos de prueba, que el modelo NUNCA ha visto.

Es importante recordar que experimentar con los hiperparámetros causa un sobreajuste de los datos de validación.

La prueba es la instancia final absoluta.  No debe llevarse a cabo la prueba antes de haber terminado con el ajuste del modelo.  Si se hacen ajustes luego de la prueba, se empieza a sobreajustar con el conjunto de datos de prueba.  Esto "tira por la borda" el propósito de la prueba. 



In [ ]:
perdida_prueba, precision_prueba = modelo.evaluate(entradas_prueba, metas_prueba)

In [ ]:
print('\nPérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Utilizando el modelo e hiperparámetros iniciales dados en este Notebook, la precisión final de la prueba debe rondar en un 80%.

Cada vez que se ejecuta el código, se obtiene una precisión distinta porque cada entrenamiento es distinto.

Intencionalmente se ha dejado en una solución sub-óptima para que se pueda tratar de edificar sobre la misma.